## 1.1-Importando librerias necesarias.

In [1]:
from gensim.models import KeyedVectors
import matplotlib.pyplot as  plt
from collections import Counter
import pandas as pd
import numpy as np
import random
import string
import math
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

/home/fedricio/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1-Dataset Puestos

In [107]:
df_Jobs = pd.read_csv("Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/Job_Descr/Job_Positions_Scrapped/10_Positions_Only_Qualif.csv")
df_Jobs = df_Jobs.rename(columns={'job_title': 'Job_Title', 'job_description':'Job_Description'})

df_Jobs

,Job_Title,Job_Description
0,Machine Learning Engineer,Leveraging the latest machine and deep learnin...
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...
2,Data Scientist,"Master’s degree or above in a STEM field, incl..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,..."
4,Full Stack Web Developer,\n\n Graduate Degree in Information Technol...
5,Full Stack Web Developer 2,\n· Enter existing website codebases and exten...
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...
7,HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...
8,Security Specialist,Work in a fast-paced environment that combine ...
9,Security Specialist 2,\n Handling incoming requests for assistanc...


### 2-Dataset CVs

### PDF a texto usando PyPDF2.

NO usar PyPDF2. Razón: Funciona para algunos archivos pdf, pero depende en el generador que se use. Por ejemplo extrae muy mal de los archivos que generé desde el Office y los guardé como pdf (por ejemplo el de 1-ML_Engineer_Bradly_Johnston).

#### Conclusión: usar pdfplumber (abajo).

In [54]:
#Leemos los CVs almacenados en nuestra carpeta y los extraemos uno por uno mediante la libreria PyPDF; devolviendonos
#una secuencia de strings.
import PyPDF2
import os
import collections
from os import listdir
from os.path import isfile, join

#pathCVs='Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs' #Ruta Relativa, ubicacion de la carpeta.  #Antes, ruta absoluta: r'C:\Users\calon\Desktop\Notebooks\Resume-Scoring-using-NLP-master\Resume-Scoring-using-NLP-master\Resumes'
pathCVs='Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs_2'
onlyfiles = [os.path.join(pathCVs, f) for f in os.listdir(pathCVs) if os.path.isfile(os.path.join(pathCVs, f))]

print("Cantidad de CVs extraidos:", len(onlyfiles))

#Funcion para extraer las palabras del CV:
import collections

def pdfextract(file):
    page_content = ""
    pdf_file = open(file, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    c = collections.Counter(range(number_of_pages))
    for i in c:
        page = read_pdf.getPage(i)
        page_content += page.extractText()
    return (page_content.encode('utf-8'))

def extract_text(file):
    text = pdfextract(file).decode('utf-8')
    return text

#Obtenemos todas las palabras del CV sin preprocesar ni nada:
df_Candidates=pd.DataFrame(columns = ['Candidate_Name','Content_CV'])
i=0
while i < len(onlyfiles):
    file=onlyfiles[i]
    base = os.path.basename(file)  #Test_Phoebe Buffay.pdf
    filename = os.path.splitext(base)[0]  #Test_Phoebe Buffay
    dat=extract_text(file)
    data = [{'Candidate_Name':filename, 'Content_CV':dat}]
    df_Candidates=df_Candidates.append(data, ignore_index=True)
    i+=1

df_Candidates

Cantidad de CVs extraidos: 11


,Candidate_Name,Content_CV
0,1-ML_Engineer_Bradly_Johnston,"\n\n\n\n˛˜#\n%&'˙%˘(+ ˛ ,-\n\n #1000\n˛#˚##˚#...."
1,3-Web_Dev_Karen_Higgins,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...
2,5-Security_Specialist_Denis Banik,"\n\n\n\n˘ˇˇˇ˚˜ˇˇ\n ˇˇˇˇˇˇˇ\nˇˇˇ""ˇ˚ˇˇ\n˚ˇˇˇ""ˇ˚\..."
3,5-Security_Specialist_Ahmed Wayne,"\n\n\nˆ˛ˆˆ\n˚!""!\n#\n\n˚!\n\n˘ˇˇ˘ˆˇ\n\n\n\n\n˘..."
4,0-ML_Engineer_Bradly_Johnston,"\n\n\n\n˛˜#\n%&'˙%˘(+ ˛ ,-\n\n #1000\n˛#˚##˚#...."
5,4-HCM_Robert_Smith,Sap Hcm Consultant\nROBERT SMITH\nPhone: (123)...
6,1-ML_Engineer_Jonathon_Price,"\n\n˘\n˚\n!""$\n $\n)'ˇ*\n$.0 $0$\n!2((---34$$(..."
7,2-Data_Scientist_Rahul_Malik,
8,2-Data_Scientist_Karla_Lewis,
9,3-Web_Dev_Alec_Dionisio,LEfhEeh[pY¿4˚LMš9VEBDM[YMfÀM[‡~–~ƒ†⁄‡•9VEBDM[Y...


### Solución: PDF a texto usando pdfplumber.

In [51]:
#Prueba: convirtiendo solo un archivo a pdf, vemos que lo extrae correctamente.
import pdfplumber
PDF_file='Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs_2/1-ML_Engineer_Bradly_Johnston.pdf'

single_page_text = ""
all_text = ""
pdf = pdfplumber.open(PDF_file)
for pdf_page in pdf.pages:
    single_page_text = pdf_page.extract_text()
    #print(single_page_text)
    all_text = all_text + '\n' + single_page_text
print(all_text)        
pdf.close()


Bradly Johnston
435 Kasey Vista, Detroit
+1 (555) 512 4119
Work Experience
Senior Software Engineer, Machine Learning
08/2016 - PRESENT
Phoenix, AZ 
• Experience of managing Algorithm development teams 
• Strong computer skills, specifically Microsoft Office, including Excel, Outlook and 
PowerPoint 
• Work with application/platform engineers and data scientists to implement and efficiently 
scale Machine Learning Platform & Applications 
• Build and enhance machine-learning platform by absorbing best-of-breed tools, techniques 
and best practices 
• Work closely with the other engineering team to improve our machine learning 
infrastructure and data quality 
• Identify new opportunities to apply machine learning to different parts of the e-commerce 
experience 
• Design and run experiments to advance our machine learning understanding and techniques 
• Use your mathematical, statistical and programmatic knowledge to search, analyze, and 
model to enhance customer experience 
• Select

In [56]:
#Uso final:

import pdfplumber
import os
import collections
from os import listdir
from os.path import isfile, join

#Leemos los CVs almacenados en nuestra carpeta y los extraemos uno por uno convirtiendolos a texto
#mediante la libreria pdfplumber:
pathCVs='Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs_2'
onlyfiles = [os.path.join(pathCVs, f) for f in os.listdir(pathCVs) if os.path.isfile(os.path.join(pathCVs, f))]
print("Cantidad de CVs extraidos:", len(onlyfiles))


#pdf = pdfplumber.open(PDF_file)
#for pdf_page in pdf.pages:
#    single_page_text = pdf_page.extract_text()
#    print(single_page_text)
#    all_text = ''
#    all_text = all_text + '\n' + single_page_text
#    print(all_text)        
#pdf.close()


#Funcion para extraer las palabras del CV:

def pdfextract(PDF_file):
    single_page_text = ""
    all_text = ""
    pdf = pdfplumber.open(PDF_file)
    for pdf_page in pdf.pages:
        single_page_text = pdf_page.extract_text()
        all_text = all_text + '\n' + single_page_text
    pdf.close()
    #return(all_text)        
    return(all_text.encode('utf-8'))

def extract_text(file):
    text = pdfextract(file).decode('utf-8')
    #text = pdfextract(file)
    return text

#Obtenemos todas las palabras del CV sin preprocesar ni nada:
df_Candidates=pd.DataFrame(columns = ['Candidate_Name','Content_CV'])
i=0
while i < len(onlyfiles):
    file=onlyfiles[i]
    base = os.path.basename(file)  #Test_Phoebe Buffay.pdf
    filename = os.path.splitext(base)[0]  #Test_Phoebe Buffay
    dat=extract_text(file)
    data = [{'Candidate_Name':filename, 'Content_CV':dat}]
    df_Candidates=df_Candidates.append(data, ignore_index=True)
    i+=1

df_Candidates

Cantidad de CVs extraidos: 11


,Candidate_Name,Content_CV
0,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+..."
1,3-Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...
2,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...
3,5-Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation..."
4,0-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+..."
5,4-HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...
6,1-ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n..."
7,2-Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...
8,2-Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...
9,3-Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io..."


**Limpieza De los CVs**:
1. Limpieza "general": Pasar todo a minúscula, eliminar signos de puntuación, espacios en blanco.
2. Limpieza de stop words.
3. Steaming para unir palabras: computer_science, machine_learning.
4. Lematización. (VER)
5. Limpieza en txt de datos personales (mails, telefonos).
6. Limpieza de palabras innecesarias en archivo txt (Page, meses, años, etc).

**Limpieza de los Puestos**:
1. Limpieza "general": Pasar todo a minúscula, eliminar signos de puntuación, espacios en blanco.
2. Limpieza de stop words.
3. Steaming para unir palabras: computer_science, machine_learning.
4. Lematización. (VER)
5. Limpieza de palabras innecesarias en archivo txt (VER: por ej. mails que dejan las empresas, telefonos, etc.).
6. Luego, para la obtención de Keywords (VER si es necesario): --> Por el problema que me tomaba las keywords como "you", "I" y demás..., podría hacer un txt con lenguajes y skills por "defecto" y me fijo si están en las keywords... sino no las pongo.


### Funciones para limpiar datos en columnas de un DF

In [57]:
import regex as re #Usado en la función remove_punctuation_and_special_characters.
from nltk.stem import WordNetLemmatizer   #Usado para lematización.
nltk.download('wordnet')
from nltk.tokenize import word_tokenize   #Usado para tokenizar.

#Definimos nuestras stop words desde nltk:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
                
def lower_text(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: x.lower() if isinstance(x,str) else x)
    
def remove_stop_words(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

def remove_punctuation_and_special_characters(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: re.sub('[^\w-]|(\d+)',' ', x) if isinstance(x,str) else x)
                                                                #Macheamos cualquier non-word excepto el -; y tambien macheamos los números (\d+)
    #Macheos:
        #'\W+'                  --> Cualquier non-word.
        #'[()!@#$/,;.:"]'       --> todos esos carácteres especiales que pongo ahi. 
        #'\w+[-]\w+'            --> machea a las palabras non-algo.
        #'[^\w-+]'              --> Machea todas las non-words excepto el guion medio y el +.
        #'\C\+\+'               --> Machea C++.
        #[^\w-+]|(\C\+\+|team)  --> Machea todas las non-words excepto el guion medio y el +. Y tambien machea C++ y la palabra "team"
        #[^\w-+]|(\C\+\+|\d+)   --> Machea todas las non-words excepto el guion medio y el +. Y tambien machea C++ y los números (\d+).
        
    #Para m.sc' --> reemplazar el . por espacio cuando antes y despues del punto esta todo "junto"... para diferenciarlo del punto de una oración. 
    
#Tokenizar es el proceso de parsear Strings de texto en diferentes secciones ("tokens"). Debemos tokenizar teniendo en cuenta qué hacer con las comas, con los espacios en blanco, 
#con los "-", etc. Para esto usamos la función "word_tokenize" que se encargará de esta tarea. 
#Esta funcion NO divide las palabras que estan con un "-"... de esta manera la palabra "scale-up" queda así y NO se divide.
#Tambien tendremos la palabra con una única letra ("c", del lenguaje c) macheado únicamente a un token "c". 
def tokenize_and_lemmatization(text_column):
    tokens = word_tokenize(text_column) #Tokenizamos.
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [wordnet_lemmatizer.lemmatize(tok).lower() for tok in tokens] # Stem words.
    return list(lemmatized_tokens)
    
def cleaning_DF(DF,column_to_clean):
    clean_column='clean_'+column_to_clean
    DF[clean_column]=DF[column_to_clean] #Copiamos el contenido de 'column_to_clean' en 'clean_column' para utilizarla en las funciones posteriores.
    lower_text(DF,clean_column)
    remove_punctuation_and_special_characters(DF,clean_column)
    remove_stop_words(DF,clean_column)
     
def tokenize_and_lemmatize(DF,column_to_clean):
    clean_column='clean_'+column_to_clean
    tokens_column='tokens_'+column_to_clean
    DF[tokens_column] = DF[clean_column].apply(tokenize_and_lemmatization)

[nltk_data] Downloading package wordnet to /home/fedricio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Limpieza de datos del DF de Jobs.

In [58]:
cleaning_DF(df_Jobs,'Job_Description')
tokenize_and_lemmatize(df_Jobs,'Job_Description')
print(df_Jobs['Job_Description'].iloc[1])  #Tomamos el job_description de la posición 1 como ejemplo.
#non-expert  debería mantenerse.
#deberia juntar el Computer Science
print("#######################################################")
print(df_Jobs['clean_Job_Description'].iloc[1]) 
print("#######################################################")
print(df_Jobs['tokens_Job_Description'].iloc[1]) 
print("#######################################################")
df_Jobs

Collaborate with a multidisciplinary team to gain insight into complex biochemical systems.
Design computational models to study various interactions such as interactions between genomes, proteins, and binding sites
Extract various features from the computational models and communicate the results back to the team.
Predict the behaviour of new protein structures on certain binding sites using the computational models.
Work with the team of software engineers to embed your models into production.
Perform other related duties in keeping with the purpose and accountabilities of the job.

M.Sc. or Ph.D. in Engineering/Computer Science, or an equivalent combination of experience and knowledge.
2+ years' experience applying machine learning and deep learning concepts to real-world problems.
Solid programming skills with a focus on writing clean/maintainable code, with 2+ years of experience in Python (preferred), Java, or C++ programming.
Good knowledge of machine learning libraries (Tensorf

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
0,Machine Learning Engineer,"BSc/MSc/PhD in Physics, Mathematics, Biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica..."
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
2,Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
4,Full Stack Web Developer,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."
5,Full Stack Web Developer 2,\n· Enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ..."
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
7,HCM Consultant 2,\nConducting Fit/Gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro..."
8,Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn..."
9,Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi..."


#### Limpieza de datos del DF de Jobs.

In [59]:
cleaning_DF(df_Candidates,'Content_CV')
tokenize_and_lemmatize(df_Candidates,'Content_CV')

df_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV
0,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
1,3-Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise..."
2,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu..."
3,5-Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation...",ahmed wayne address abu dhabi uae nationality ...,"[ahmed, wayne, address, abu, dhabi, uae, natio..."
4,0-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
5,4-HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...,sap hcm consultant phone email info qwikresume...,"[sap, hcm, consultant, phone, email, info, qwi..."
6,1-ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n...",jonathon price terry groves boston experience ...,"[jonathon, price, terry, grove, boston, experi..."
7,2-Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...,rahul malik nlp data scientist contact work ex...,"[rahul, malik, nlp, data, scientist, contact, ..."
8,2-Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,..."
9,3-Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io...",chestertown md hi alecdionis io alecdionis io ...,"[chestertown, md, hi, alecdionis, io, alecdion..."


In [62]:
df_Candidates.iloc[10]['tokens_Content_CV']

['calonge',
 'federico',
 'matías',
 'hcm',
 'technical',
 'consultant',
 'federico',
 'working',
 'oracle',
 'tool',
 'since',
 'october',
 'march',
 'participated',
 'erp',
 'cloud',
 'project',
 'performing',
 'reporting',
 'task',
 'working',
 'module',
 'ap',
 'ar',
 'gl',
 'since',
 'march',
 'participating',
 'hcm',
 'cloud',
 'project',
 'performing',
 'reporting',
 'extraction',
 'integration',
 'task',
 'working',
 'module',
 'core',
 'recruitment',
 'since',
 'october',
 'member',
 'artificial',
 'intelligence',
 'ai',
 'committee',
 'oracle',
 'strong',
 'interest',
 'data',
 'science',
 'machine',
 'learning',
 'last',
 'year',
 'computer',
 'engineering',
 'degree',
 'studying',
 'th',
 'year',
 'english',
 'personal',
 'academic',
 'programming',
 'project',
 'http',
 'github',
 'com',
 'federicocalonge',
 'skill',
 'summary',
 'oracle',
 'hcm',
 'cloud',
 'core',
 'recruitment',
 'oracle',
 'erp',
 'cloud',
 'account',
 'receivable',
 'ar',
 'account',
 'payable',
 'ap'

### AGREGADO: Funciones para tokenizar textos de una columna de un DF

In [63]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import string
#Esto de arriba se descarga LOCALMENTE en home/user/nltk_data.

def tokenize(text):
    #Convierte el texto recibido como entrada en una lista de tokens (palabras).
    #Dentro pasamos todo a minusculas, se eliminan signos de puntuación, espacios en blanco y se eliminan las stop words.
        #input  --> text: el documento como un STRING. 
        #output --> texts_tokens: cada palabra del texto en una LISTA de string. 
    texts_tokens=[]
    for line in text:
        tokens=word_tokenize(line)
        tok=[w.lower() for w in tokens]
        table=str.maketrans('','',string.punctuation)
        strpp=[w.translate(table) for w in tok]
        words=[word for word in strpp if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        words=[w for w in words if not w in stop_words]
        texts_tokens.append(words)
    return texts_tokens

def text_to_vector(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.

[nltk_data] Downloading package punkt to /home/fedricio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
#PARA PROBAR LO DE ARRIBA COMO HACIA ANTES:
#Tomamos como ejemplo la descripción del primer puesto del DF "df_DS_Jobs":
text_description_1st_job = df_Jobs.iloc[1]['Job_Description']
print(text_description_1st_job)

#Usamos la funcion text_to_vector() para text description:
#vector_test_description = text_to_vector([text_description_1st_job])  #Recordar mandar lo de adentro como una LISTA.
#vector_test_description

Collaborate with a multidisciplinary team to gain insight into complex biochemical systems.
Design computational models to study various interactions such as interactions between genomes, proteins, and binding sites
Extract various features from the computational models and communicate the results back to the team.
Predict the behaviour of new protein structures on certain binding sites using the computational models.
Work with the team of software engineers to embed your models into production.
Perform other related duties in keeping with the purpose and accountabilities of the job.

M.Sc. or Ph.D. in Engineering/Computer Science, or an equivalent combination of experience and knowledge.
2+ years' experience applying machine learning and deep learning concepts to real-world problems.
Solid programming skills with a focus on writing clean/maintainable code, with 2+ years of experience in Python (preferred), Java, or C++ programming.
Good knowledge of machine learning libraries (Tensorf

###  1.3.1.1-Agregando columna 'keywords' a nuestro DF obteniendolas mediante keyword extraction.

Para esto ver previamente el ejemplo general (ejemplo número 5) del Notebook '1-WMD_And_Cosine_Similarity_General'.

#### Resumen: 
Document -> Remove stop words -> Find Term Frequency (TF) -> Find Inverse Document Frequency (IDF) -> Find TF*IDF -> Get top N Keywords

Definimos las funciones principales a utilizar:

In [65]:
from nltk import tokenize
from operator import itemgetter
import math

def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result
    
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))
   
def get_keywords(doc,n):
    from nltk import tokenize
    from operator import itemgetter
    import math

    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize 
    #stop_words = set(stopwords.words('english'))

    total_words = doc.split()
    total_word_length = len(total_words)
    # print(total_word_length)
    
    total_sentences = tokenize.sent_tokenize(doc)
    total_sent_len = len(total_sentences)
    print(total_sentences)
    print(total_sent_len)
    
    #Calculo de TF:
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        #if each_word not in stop_words:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    
    #Calculo de IDF:
    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        #if each_word not in stop_words:
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)

    #Calculo de TF * IDF:
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    # print(tf_idf_score)

    #Obtenemos las 5 palabras top:
    n_palabras_top = get_top_n(tf_idf_score, n)
    # print(n_palabras_top)
    
    #Obtenemos la lista de keywords:
    keyword_list = list(n_palabras_top.keys())   #n_palabras_top es un dic.
    
    return keyword_list

In [66]:
#Ejemplo para comprobar el correcto funcionamiento de las funciones anteriores:
doc_1 = "Computer programming is the process of designing and building an executable computer program to accomplish a specific computing result or to perform a specific task. Programming involves tasks such as: analysis, generating algorithms, profiling algorithms' accuracy and resource consumption, and the implementation of algorithms in a chosen programming language (commonly referred to as coding).[1][2] The source code of a program is written in one or more languages that are intelligible to programmers, rather than machine code, which is directly executed by the central processing unit. The purpose of programming is to find a sequence of instructions that will automate the performance of a task (which can be as complex as an operating system) on a computer, often for solving a given problem. Proficient programming thus often requires expertise in several different subjects, including knowledge of the application domain, specialized algorithms, and formal logic. Tasks accompanying and related to programming include: testing, debugging, source code maintenance, implementation of build systems, and management of derived artifacts, such as the machine code of computer programs. These might be considered part of the programming process, but often the term software development is used for this larger process with the term programming, implementation, or coding reserved for the actual writing of code. Software engineering combines engineering techniques with software development practices. Reverse engineering is a related process used by designers, analysts and programmers to understand and re-create/re-implement."
doc = doc_1.lower()
#'I am a graduate. I want to learn Python. I like learning Python. Python is easy. Python is interesting. Learning increases thinking. Everyone should invest time in learning'
keyword_list = get_keywords(doc,5)  #Le pasamos el número de keywords que queremos y el documento/string.
#dic.keys()
keyword_list

['computer programming is the process of designing and building an executable computer program to accomplish a specific computing result or to perform a specific task.', "programming involves tasks such as: analysis, generating algorithms, profiling algorithms' accuracy and resource consumption, and the implementation of algorithms in a chosen programming language (commonly referred to as coding).", '[1][2] the source code of a program is written in one or more languages that are intelligible to programmers, rather than machine code, which is directly executed by the central processing unit.', 'the purpose of programming is to find a sequence of instructions that will automate the performance of a task (which can be as complex as an operating system) on a computer, often for solving a given problem.', 'proficient programming thus often requires expertise in several different subjects, including knowledge of the application domain, specialized algorithms, and formal logic.', 'tasks acco

['designing', 'building', 'executable', 'accomplish', 'computing']

In [67]:
#Ahora aplicamos la funcion get_keywords(n,doc) a nuestro DF para generar la columna 'keywords':
df_Jobs

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
0,Machine Learning Engineer,"BSc/MSc/PhD in Physics, Mathematics, Biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica..."
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
2,Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
4,Full Stack Web Developer,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."
5,Full Stack Web Developer 2,\n· Enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ..."
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
7,HCM Consultant 2,\nConducting Fit/Gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro..."
8,Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn..."
9,Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi..."


In [68]:
df_Jobs['Job_Description'] = df_Jobs['Job_Description'].apply(lambda x: x.lower() if isinstance(x,str) else x)


df_Jobs['Keywords'] = df_Jobs['Job_Description'].apply(get_keywords, args=[10])
df_Jobs

['bsc/msc/phd in physics, mathematics, biomedical/software engineering or computer science.', '3+ years of experience developing image processing / machine learning products and pipelines.', 'advantage\nfamiliarity with web development (e.g., flask, serverless) and deployment (e.g., docker) and cloud services (e.g., aws).', 'entrepreneurial experience and/or passion for working in small teams.', 'peer-reviewed academic publications.']
5
['collaborate with a multidisciplinary team to gain insight into complex biochemical systems.', 'design computational models to study various interactions such as interactions between genomes, proteins, and binding sites\nextract various features from the computational models and communicate the results back to the team.', 'predict the behaviour of new protein structures on certain binding sites using the computational models.', 'work with the team of software engineers to embed your models into production.', 'perform other related duties in keeping wit

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords
0,Machine Learning Engineer,"bsc/msc/phd in physics, mathematics, biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica...","[(eg,, experience, bsc/msc/phd, physics,, math..."
1,Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","[2+, knowledge, of, with, experience, collabor..."
2,Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","[master’s, degree, or, above, in, a, stem, fie..."
3,Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","[projects, techniques, skills, organization’s,..."
4,Full Stack Web Developer,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","[experience, written, typescript), skills, and..."
5,Full Stack Web Developer 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ...","[5, (required), enter, existing, website, code..."
6,HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","[requirements, experience, work, consulting, c..."
7,HCM Consultant 2,\nconducting fit/gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro...","[and, experience, business, management,, abili..."
8,Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","[experience, with, work, fast-paced, combine, ..."
9,Security Specialist 2,\n handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi...","[security, of, handling, incoming, requests, f..."


In [69]:
#Transformamos la lista de strings de la columna Keywords en strings separados por ', ':
df_Jobs['Keywords'] = [', '.join(map(str, l)) for l in df_Jobs['Keywords']]

#Minusculas:
df_Jobs['Keywords'] = df_Jobs['Keywords'].apply(lambda x: x.lower() if isinstance(x,str) else x) #Ponemos todas en minusculas (importante para hacer las comparaciones futuras).
df_Jobs['Job_Description'] = df_Jobs['Job_Description'].apply(lambda x: x.lower() if isinstance(x,str) else x) #Tambien en minuscula el Job_Description.

df_Jobs

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords
0,Machine Learning Engineer,"bsc/msc/phd in physics, mathematics, biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica...","(eg,, experience, bsc/msc/phd, physics,, mathe..."
1,Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora..."
2,Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel..."
3,Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ..."
4,Full Stack Web Developer,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,..."
5,Full Stack Web Developer 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ...","5, (required), enter, existing, website, codeb..."
6,HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co..."
7,HCM Consultant 2,\nconducting fit/gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro...","and, experience, business, management,, abilit..."
8,Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o..."
9,Security Specialist 2,\n handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi...","security, of, handling, incoming, requests, fo..."


# VERSION 2 TF IDF.

In [70]:
from nltk import tokenize
from nltk.tokenize import word_tokenize 
from operator import itemgetter
import math

def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result
    
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

def lower_text_2(text):
    new_text = text.lower()
    return new_text

def remove_punctuation_and_special_characters_2(text):
    new_text = re.sub('[^\w-.!?]|(\d+)',' ', text)
    return new_text
    #Todo lo que machee lo reemplaza por un espacio. Machea:
        #1-Todas las non-words excepto el guion medio (-), el punto (.), signo de interrogación final (?) y signo de exclamación final (!). 
        #2-Y los números.

def tokenize_and_lemmatization_2(total_words):
    wordnet_lemmatizer = WordNetLemmatizer()
    text=''
    for each_word in total_words:
        word_lemmatized = wordnet_lemmatizer.lemmatize(each_word).lower()
        text = text+word_lemmatized+' '
    return text

def get_keywords_2(doc,n):
    #Vamos a considerar a nuestro 'doc' como el total de todas nuestras palabras de la celda 'Job Description'.
    #Para calcular el IDF dividimos nuestro 'doc' en 'sentences', donde cada 'sentence' es una oración del la columna 'Job Description'.
    #Para realizar esta separación de oraciones utilizamos la función sent_tokenize (si detecta un punto (.), signo de interrogación final (?) o signo de exclamación final (!) se considera que termina una oración).
    
    # 1-Obtenemos nuestro doc y realizamos la limpieza del mismo:
    doc = remove_punctuation_and_special_characters_2(doc)
    doc = lower_text_2(doc)
    total_words = doc.split()
    doc = tokenize_and_lemmatization_2(total_words)
    print(doc)
    #Realizamos nuevamente el split, ya que algunas palabras cambiaron por la lematización:
    total_words = doc.split()
    total_word_length = len(total_words)
    
    # 2-Realizamos la división por oraciones:
    total_sentences = tokenize.sent_tokenize(doc)
    total_sent_len = len(total_sentences)
    print(total_sentences)
    print(total_sent_len)
    
    #Calculo de TF:
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        each_word = each_word.replace('?','')
        each_word = each_word.replace('!','')
        print(each_word)
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    
    #Calculo de IDF:
    idf_score = {}
    for each_word in total_words:
        #Para eliminar los puntos (.), signos interrogación final (?) y signos de exclamación final (!) de las palabras que expresan fin de oración 
        #(por ejemplo en "Hola Pepe, como estás? Yo bien, gracias." --> reemplazaría el "estás?" por "estás" y el "gracias." por "gracias"). 
        each_word = each_word.replace('.','')
        each_word = each_word.replace('?','')
        each_word = each_word.replace('!','')
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)

    #Calculo de TF * IDF:
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    # print(tf_idf_score)

    #Obtenemos las 5 palabras top:
    n_palabras_top = get_top_n(tf_idf_score, n)
    # print(n_palabras_top)
    
    #Obtenemos la lista de keywords:
    keyword_list = list(n_palabras_top.keys())   #n_palabras_top es un dic.
    
    return keyword_list

In [71]:
#Ejemplo para comprobar el correcto funcionamiento de las funciones anteriores:
doc_1 = "Python is fast! Can you believe? ß Computers programming."
#is the process of designing and building an executable computer program to accomplish a specific computing result or to perform a specific task. Programming involves tasks such as: analysis, generating algorithms, profiling algorithms' accuracy and resource consumption, and the implementation of algorithms in a chosen programming language (commonly referred to as coding).[1][2] The source code of a program is written in one or more languages that are intelligible to programmers, rather than machine code, which is directly executed by the central processing unit. The purpose of programming is to find a sequence of instructions that will automate the performance of a task (which can be as complex as an operating system) on a computer, often for solving a given problem. Proficient programming thus often requires expertise in several different subjects, including knowledge of the application domain, specialized algorithms, and formal logic. Tasks accompanying and related to programming include: testing, debugging, source code maintenance, implementation of build systems, and management of derived artifacts, such as the machine code of computer programs. These might be considered part of the programming process, but often the term software development is used for this larger process with the term programming, implementation, or coding reserved for the actual writing of code. Software engineering combines engineering techniques with software development practices. Reverse engineering is a related process used by designers, analysts and programmers to understand and re-create/re-implement."
doc = doc_1.lower()
#'I am a graduate. I want to learn Python. I like learning Python. Python is easy. Python is interesting. Learning increases thinking. Everyone should invest time in learning'
keyword_list = get_keywords_2(doc,5)  #Le pasamos el número de keywords que queremos y el documento/string.
#dic.keys()
keyword_list

python is fast! can you believe? ß computer programming. 
['python is fast!', 'can you believe?', 'ß computer programming.']
3
python
is
fast
can
you
believe
ß
computer
programming


['python', 'is', 'fast', 'can', 'you']

In [72]:
#df_Jobs['Keywords'] = df_Jobs['Job_Description'].apply(get_keywords_2, args=[3])
#df_Jobs

#df_Jobs['Keywords'] = df_Jobs['Job_Description'].apply(get_keywords_2, args=[3])

text_description_1st_job = df_Jobs.iloc[1]['Job_Description']
print(text_description_1st_job)

keyword_list = get_keywords_2(text_description_1st_job,30)  #Le pasamos el número de keywords que queremos y el documento/string.
#dic.keys()
keyword_list

collaborate with a multidisciplinary team to gain insight into complex biochemical systems.
design computational models to study various interactions such as interactions between genomes, proteins, and binding sites
extract various features from the computational models and communicate the results back to the team.
predict the behaviour of new protein structures on certain binding sites using the computational models.
work with the team of software engineers to embed your models into production.
perform other related duties in keeping with the purpose and accountabilities of the job.

m.sc. or ph.d. in engineering/computer science, or an equivalent combination of experience and knowledge.
2+ years' experience applying machine learning and deep learning concepts to real-world problems.
solid programming skills with a focus on writing clean/maintainable code, with 2+ years of experience in python (preferred), java, or c++ programming.
good knowledge of machine learning libraries (tensorf

['knowledge',
 'of',
 'with',
 'project',
 'experience',
 'collaborate',
 'multidisciplinary',
 'gain',
 'insight',
 'complex',
 'biochemical',
 'systems',
 'study',
 'between',
 'genome',
 'extract',
 'feature',
 'from',
 'communicate',
 'back',
 'predict',
 'behaviour',
 'new',
 'structure',
 'certain',
 'using',
 'models',
 'work',
 'software',
 'engineer']

# VERSION 3 TF-IDF:

Ahora en lugar de considerar al DOC como solo la celda de 'Description', ahora consideramos TODAS las celdas de 'Description'. Asi, cada documento es una celda del job_description.

El problema con la versión 2 es que generalmente en los jobs description no ponen "." para finalizar cada oraciones, entonces me toma todo el texto como 1 sola oración y al ahcer el IDF no tiene sentido porque solo tengo 1 oración.

Cree una variable "text" con TODOS los textos. Y saco el keyword con todos esos textos partiendo en "sentences". Considero como caracter de fin de oración a 'ß'. Y luego si queremos las keywords de 1 job description en particular nos fijamos solo las keywords que aparezcan en este texto (esto lo hacemos en get_top_n).

In [73]:
from nltk import tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktLanguageVars
class BulletPointLangVars(PunktLanguageVars):
    sent_end_chars = ('ß')
tokenizer = PunktSentenceTokenizer(lang_vars = BulletPointLangVars())

from nltk.tokenize import word_tokenize 
from operator import itemgetter
import math

def get_top_n(dict_elem, text_description_1st_job, n):
    # dict_elem.items() son diccionarios con key=palabra y value=valor tf-idf--> ('bsc', 0.0009448441087378112)
    for key in dict_elem.copy():
        if key not in text_description_1st_job:
             #del dict_elem[key]
            dict_elem.pop(key)
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result
    
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

def lower_text_2(text):
    new_text = text.lower()
    return new_text

def remove_punctuation_and_special_characters_2(text):
    new_text = re.sub('[^\w-ß]|(\d+)',' ', text)
    return new_text
    #Todo lo que machee lo reemplaza por un espacio. Machea:
        #1-Todas las non-words excepto el guion medio (-) y el ß.
        #2-Y los números.

def tokenize_and_lemmatization_2(total_words):
    wordnet_lemmatizer = WordNetLemmatizer()
    text=''
    for each_word in total_words:
        word_lemmatized = wordnet_lemmatizer.lemmatize(each_word).lower()
        text = text+word_lemmatized+' '
    return text

def get_keywords_2(doc,text_description_1st_job,n):
    #Vamos a considerar a nuestro 'doc' como el total de todas nuestras palabras de la celda 'Job Description'.
    #Para calcular el IDF dividimos nuestro 'doc' en 'sentences', donde cada 'sentence' es una oración del la columna 'Job Description'.
    #Para realizar esta separación de oraciones utilizamos la función sent_tokenize (si detecta un punto (.), signo de interrogación final (?) o signo de exclamación final (!) se considera que termina una oración).
    
    # 1-Obtenemos nuestro doc y realizamos la limpieza del mismo:
    doc = remove_punctuation_and_special_characters_2(doc)
    doc = lower_text_2(doc)
    total_words = doc.split()
    doc = tokenize_and_lemmatization_2(total_words)
    #print(doc)
    #Realizamos nuevamente el split, ya que algunas palabras cambiaron por la lematización:
    total_words = doc.split()
    total_word_length = len(total_words)
    
    # 2-Realizamos la división por oraciones:
    total_sentences = tokenizer.tokenize(doc)
    total_sent_len = len(total_sentences)
    #print(total_sentences)
    #print(total_sent_len)
    
    #Calculo de TF:
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('ß','')
        #print(each_word)
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    
    #Calculo de IDF:
    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('ß','')
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)

    #Calculo de TF * IDF:
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    # print(tf_idf_score)

    #Obtenemos las 5 palabras top:
    n_palabras_top = get_top_n(tf_idf_score, text_description_1st_job, n)
    #print(n_palabras_top)
    
    #Obtenemos la lista de keywords:
    keyword_list = list(n_palabras_top.keys())   #n_palabras_top es un dic.
    
    return keyword_list

In [77]:
text_description_1st_job = df_Jobs.iloc[1]['tokens_Job_Description'] #COMPARAMOS CON ESTA, NO CON JOB_DESCRIPTION YA QUE AHI ESTAN LAS PALABRAS SIN LEMATIZAR ni aplicar ninguna limpieza.

text = ' ß '.join(df_Jobs.Job_Description)
text

#Vamos a obtener las 20 keyword SOLO si estan en el documento 1 (o sea el 1er job description).
keyword_list = get_keywords_2(text,text_description_1st_job,10)  #Le pasamos el número de keywords que queremos y el documento/string.
keyword_list

['multidisciplinary',
 'gain',
 'biochemical',
 'genome',
 'extract',
 'back',
 'predict',
 'behaviour',
 'structure',
 'certain']

In [76]:
#HACECR: Aplicando esto a la columna 'Keywords':

#df_Jobs['Job_Description'] = df_Jobs['Job_Description'].apply(lambda x: x.lower() if isinstance(x,str) else x)
#df_Jobs['Keywords'] = df_Jobs['Job_Description'].apply(get_keywords_2, args=[10])
#df_Jobs

#Transformamos la lista de strings de la columna Keywords en strings separados por ', ':
#df_Jobs['Keywords'] = [', '.join(map(str, l)) for l in df_Jobs['Keywords']]

#Minusculas:
#df_Jobs['Keywords'] = df_Jobs['Keywords'].apply(lambda x: x.lower() if isinstance(x,str) else x) #Ponemos todas en minusculas (importante para hacer las comparaciones futuras).
#df_Jobs['Job_Description'] = df_Jobs['Job_Description'].apply(lambda x: x.lower() if isinstance(x,str) else x) #Tambien en minuscula el Job_Description.

#df_Jobs

In [75]:
df_Jobs

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords
0,Machine Learning Engineer,"bsc/msc/phd in physics, mathematics, biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica...","(eg,, experience, bsc/msc/phd, physics,, mathe..."
1,Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora..."
2,Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel..."
3,Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ..."
4,Full Stack Web Developer,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,..."
5,Full Stack Web Developer 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ...","5, (required), enter, existing, website, codeb..."
6,HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co..."
7,HCM Consultant 2,\nconducting fit/gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro...","and, experience, business, management,, abilit..."
8,Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o..."
9,Security Specialist 2,\n handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi...","security, of, handling, incoming, requests, fo..."


In [79]:
df_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV
0,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
1,3-Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise..."
2,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu..."
3,5-Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation...",ahmed wayne address abu dhabi uae nationality ...,"[ahmed, wayne, address, abu, dhabi, uae, natio..."
4,0-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
5,4-HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...,sap hcm consultant phone email info qwikresume...,"[sap, hcm, consultant, phone, email, info, qwi..."
6,1-ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n...",jonathon price terry groves boston experience ...,"[jonathon, price, terry, grove, boston, experi..."
7,2-Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...,rahul malik nlp data scientist contact work ex...,"[rahul, malik, nlp, data, scientist, contact, ..."
8,2-Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,..."
9,3-Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io...",chestertown md hi alecdionis io alecdionis io ...,"[chestertown, md, hi, alecdionis, io, alecdion..."


# ---------------------------------------------------------------

## 1.4- Realizando Comparaciones.

### 1.4.1- Usamos el Word2vec descargado.

In [78]:
#Link del cual descargamos el archivo: https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
EMBEDDING_FILE = '/home/fedricio/Desktop/Embeddings_Utilizados/Word2vec/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

###  1.4.2- Funciones necesarias para Cosine Sim y WDM

In [80]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
#Esto de arriba se descarga LOCALMENTE en home/user/nltk_data.

[nltk_data] Downloading package punkt to /home/fedricio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
def tokenize(text):
    #Convierte el texto recibido como entrada en una lista de tokens (palabras).
    #Dentro pasamos todo a minusculas, se eliminan signos de puntuación, espacios en blanco y se eliminan las stop words.
        #input  --> text: el documento como un STRING. 
        #output --> texts_tokens: cada palabra del texto en una LISTA de string. 
    texts_tokens=[]
    for line in text:
        tokens=word_tokenize(line)
        tok=[w.lower() for w in tokens]
        table=str.maketrans('','',string.punctuation)
        strpp=[w.translate(table) for w in tok]
        words=[word for word in strpp if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        words=[w for w in words if not w in stop_words]
        texts_tokens.append(words)
    return texts_tokens

def text_to_vector(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.

def get_cosine(doc1, doc2):
    #Get the cosine similarity between two documents.
    #Depends on the angle between two non zero vectors which are constructed by each word frequency in the two documents.
        #input  --> doc1: the first document as STRING.
                    #doc2: the second document as STRING.
        #output --> cosine similarity score

    vec1 = text_to_vector([doc1])
    vec2 = text_to_vector([doc2])
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    
def WMD(doc1,doc2):
      #Preprocess the document first and remove english stopwords then call the function that calculates the word mover distance
      #input --> doc1: the first document as STRING.
                 #doc2: the second document as STRING.
      #output --> Word Mover's Distance score
    #print(doc1)
    first_doc = tokenize([doc1])[0]     #[0] porque es una lista dentro de otra lista.
    #print(first_doc)
    second_doc = tokenize([doc2])[0]    #[0] porque es una lista dentro de otra lista.
    return (word2vec.wmdistance(first_doc, second_doc))  #'wmdistance' return the word mover distance between two documents. 

#### Test para comprobar el funcionamiento de la función text_to_vector

In [82]:
#Tomamos como ejemplo la descripción del primer puesto del DF "df_DS_Jobs":
text_description_1st_job = df_Jobs.iloc[0]['clean_Job_Description']
text_description_1st_job

'bsc msc phd physics mathematics biomedical software engineering computer science years experience developing image processing machine learning products pipelines advantage familiarity web development e g flask serverless deployment e g docker cloud services e g aws entrepreneurial experience passion working small teams peer-reviewed academic publications'

In [83]:
#Usamos la funcion text_to_vector() para text description:
vector_test_description = text_to_vector([text_description_1st_job])  #Recordar mandar lo de adentro como una LISTA.
vector_test_description

Counter({'bsc': 1,
         'msc': 1,
         'phd': 1,
         'physics': 1,
         'mathematics': 1,
         'biomedical': 1,
         'software': 1,
         'engineering': 1,
         'computer': 1,
         'science': 1,
         'years': 1,
         'experience': 2,
         'developing': 1,
         'image': 1,
         'processing': 1,
         'machine': 1,
         'learning': 1,
         'products': 1,
         'pipelines': 1,
         'advantage': 1,
         'familiarity': 1,
         'web': 1,
         'development': 1,
         'e': 3,
         'g': 3,
         'flask': 1,
         'serverless': 1,
         'deployment': 1,
         'docker': 1,
         'cloud': 1,
         'services': 1,
         'aws': 1,
         'entrepreneurial': 1,
         'passion': 1,
         'working': 1,
         'small': 1,
         'teams': 1,
         'peerreviewed': 1,
         'academic': 1,
         'publications': 1})

###  1.4.3- Comparaciones.

Las comparaciones a realizar son:

    1-Entre el contenido de los CVs y las descripciones de los puestos.
    2-Entre el contenido de los CVs y las keywords de puestos obtenidas mediante Keyword extraction.
    3-Entre el contenido de los CVs y keywords de puestos puestas "a mano" /personalizadas.

#### 1.4.3.1-Entre el contenido de los CVs y las descripciones de los puestos.

In [84]:
df_Jobs

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords
0,Machine Learning Engineer,"bsc/msc/phd in physics, mathematics, biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica...","(eg,, experience, bsc/msc/phd, physics,, mathe..."
1,Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora..."
2,Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel..."
3,Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ..."
4,Full Stack Web Developer,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,..."
5,Full Stack Web Developer 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ...","5, (required), enter, existing, website, codeb..."
6,HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co..."
7,HCM Consultant 2,\nconducting fit/gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro...","and, experience, business, management,, abilit..."
8,Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o..."
9,Security Specialist 2,\n handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi...","security, of, handling, incoming, requests, fo..."


In [85]:
df_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV
0,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
1,3-Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise..."
2,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu..."
3,5-Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation...",ahmed wayne address abu dhabi uae nationality ...,"[ahmed, wayne, address, abu, dhabi, uae, natio..."
4,0-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
5,4-HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...,sap hcm consultant phone email info qwikresume...,"[sap, hcm, consultant, phone, email, info, qwi..."
6,1-ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n...",jonathon price terry groves boston experience ...,"[jonathon, price, terry, grove, boston, experi..."
7,2-Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...,rahul malik nlp data scientist contact work ex...,"[rahul, malik, nlp, data, scientist, contact, ..."
8,2-Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,..."
9,3-Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io...",chestertown md hi alecdionis io alecdionis io ...,"[chestertown, md, hi, alecdionis, io, alecdion..."


In [86]:
#Creamos un nuevo DF concatenando los 2 dataframe de manera que se comparen todos los CVs con todos los Jobs:
#Nos quedarán 3 x 6 = 18 filas.
df_Jobs_and_Candidates = pd.merge(df_Candidates.assign(A=1), df_Jobs.assign(A=1), on='A').drop('A', 1)
df_Jobs_and_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords
0,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Machine Learning Engineer,"bsc/msc/phd in physics, mathematics, biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica...","(eg,, experience, bsc/msc/phd, physics,, mathe..."
1,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora..."
2,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel..."
3,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ..."
4,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Full Stack Web Developer,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,..."
...,...,...,...,...,...,...,...,...,...
105,4-HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co...",Full Stack Web Developer 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ...","5, (required), enter, existing, website, codeb..."
106,4-HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co...",HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co..."
107,4-HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co...",HCM Consultant 2,\nconducting fit/gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro...","and, experience, business, management,, abilit..."
108,4-HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co...",Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o..."


In [87]:
#Como prueba obtenemos el WMD y Cosine Sim comparando para la 1ra fila (entre las columnas Content_CV y Job_Description)
first_job_descr = df_Jobs_and_Candidates.loc[0,'clean_Job_Description']
first_content_cv = df_Jobs_and_Candidates.loc[0,'clean_Content_CV']

cosine_result = round(get_cosine(first_job_descr,first_content_cv),3)
wmd_result = round(WMD(first_job_descr,first_content_cv),3)
print(cosine_result)
print(wmd_result)
 
similarity_wdm = round((1/(1+wmd_result)),3)  #similarity = 1 / (1 + distance) https://groups.google.com/g/gensim/c/-pRZnsOEaPQ 
print(similarity_wdm)

0.222
1.026
0.494


In [88]:
#Aplicamos las funciones get_cosine y WMD para TODO el DF... entre el contenido del CV y la descripcion del 
#puesto; y el resultado lo guardamos en  las columnas 'Cosine_Job_Desc' y 'WMD_Job_Desc'.

import time

inicio_cosine = time.time()
df_Jobs_and_Candidates['Cosine_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round(get_cosine(row['clean_Content_CV'],row['clean_Job_Description']),3), axis=1)
fin_cosine = time.time()
print(fin_cosine-inicio_cosine)

inicio_WMD = time.time()
#df_Jobs_and_Candidates['WMD_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round(WMD(row['Content_CV'],row['Job_Description']),3), axis=1)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates['WMD_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round((1/(1+(WMD(row['clean_Content_CV'],row['clean_Job_Description'])))),3), axis=1)
fin_WMD = time.time()
print(fin_WMD-inicio_WMD)

df_Jobs_and_Candidates

0.496309757232666
110.308260679245


,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords,Cosine_Job_Desc,WMD_Job_Desc
0,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Machine Learning Engineer,"bsc/msc/phd in physics, mathematics, biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica...","(eg,, experience, bsc/msc/phd, physics,, mathe...",0.222,0.494
1,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora...",0.527,0.553
2,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel...",0.408,0.546
3,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ...",0.501,0.551
4,1-ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work...",Full Stack Web Developer,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,...",0.226,0.497
...,...,...,...,...,...,...,...,...,...,...,...
105,4-HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co...",Full Stack Web Developer 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ...","5, (required), enter, existing, website, codeb...",0.111,0.490
106,4-HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co...",HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co...",0.370,0.495
107,4-HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co...",HCM Consultant 2,\nconducting fit/gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro...","and, experience, business, management,, abilit...",0.099,0.477
108,4-HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co...",Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o...",0.138,0.486


In [105]:
is_Bradly_Johnston =  df_Jobs_and_Candidates['Candidate_Name']=='5-Security_Specialist_Denis Banik'
new_DF = df_Jobs_and_Candidates[is_Bradly_Johnston]
print(new_DF.shape)
new_DF

(10, 11)


,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords,Cosine_Job_Desc,WMD_Job_Desc
20,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Machine Learning Engineer,"bsc/msc/phd in physics, mathematics, biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica...","(eg,, experience, bsc/msc/phd, physics,, mathe...",0.065,0.473
21,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora...",0.131,0.494
22,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel...",0.088,0.491
23,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ...",0.103,0.490
24,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Full Stack Web Developer,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,...",0.128,0.486
25,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Full Stack Web Developer 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ...","5, (required), enter, existing, website, codeb...",0.094,0.489
26,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co...",0.108,0.490
27,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",HCM Consultant 2,\nconducting fit/gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro...","and, experience, business, management,, abilit...",0.156,0.497
28,5-Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o...",0.3

#### 1.4.3.2-Entre el contenido de los CVs y las keywords de puestos obtenidas mediante Keyword extraction.

In [ ]:
#APlicamos las funciones get_cosine y WMD entre el contenido del CV y las keywords que obtuvimos previamente 
#del puesto y el resultado lo guardamos en  las columnas 'Cosine_Job_Keywords' y 'WMD_Job_Keywords'.
df_Jobs_and_Candidates['Cosine_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round(get_cosine(row['clean_Content_CV'],row['Keywords']),3), axis=1)
#df_Jobs_and_Candidates['WMD_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round(WMD(row['Content_CV'],row['Keywords']),3), axis=1)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates['WMD_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round((1/(1+(WMD(row['clean_Content_CV'],row['Keywords'])))),3), axis=1)

df_Jobs_and_Candidates

#### 1.4.3.3-Entre el contenido de los CVs y keywords de puestos puestas "a mano" / personalizadas.

In [89]:
Content_CV_Phoebe_Buffay = df_Jobs_and_Candidates.loc[17,'clean_Content_CV']
print(Content_CV_Phoebe_Buffay)

karen higgins b developer areas expertise personal summary ambitious problem solver passion online businesses search engine optimisation would like join team like-minded developers karen much experience creating logical innovative solutions complex problems identifying technical thorough precise everything keen interest solutions technology mobile applications user experience someone takes responsibility personal development continually evaluating client facing skills upgrading skills stays cutting edge web development natural problem solver proven successfully completing projects consultancies software houses web troubleshooting design agencies departments javascript right karen looking suitable position company outstanding performance recognised work variety web graphi cs high profile projects bug fixing career history web design company - birmingham building tem plates web developer jul - present image cropping responsible working range projects designing appealing websites interact

In [32]:
#Agregamos 2 filas más al DF con las keywords personalizadas (las segundas keywords no tendrán nada que ver
#al contenido del CV). Y luego aplicamos sim cosine y WDM para estos casos y vemos los resultados.

df_Jobs_and_Candidates.loc[18] = ['Test',Content_CV_Phoebe_Buffay,'','','data science, machine learning, pandas, python, sql','','','',''] 
#Es lo mismo poner las Keywords con o sin comas (da igual el resultado, ya que el procesamiento se encarga de esto). 
df_Jobs_and_Candidates.loc[19] = ['Test',Content_CV_Phoebe_Buffay,'','','football, tennis, spanish','','','',''] 

df_Jobs_and_Candidates.tail()

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
#TEST: Usamos la funcion text_to_vector() para las keywords:
vector_test_description = text_to_vector(df_Jobs_and_Candidates.iloc[18]['Keywords'])
vector_test_description

#Ahora si aplicamos sim cosine y WDM para estos casos y vemos los resultados. 
df_Jobs_and_Candidates.loc[18,'Cosine_Job_Keywords'] = round(get_cosine(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords']),3)

#df_Jobs_and_Candidates.loc[18,'WMD_Job_Keywords'] = round(WMD(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords']),3)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates.loc[18,'WMD_Job_Keywords'] = round((1/(1+(WMD(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords'])))),3)

df_Jobs_and_Candidates.loc[19,'Cosine_Job_Keywords'] = round(get_cosine(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords']),3)

#df_Jobs_and_Candidates.loc[19,'WMD_Job_Keywords'] = round(WMD(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords']),3)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates.loc[19,'WMD_Job_Keywords'] = round((1/(1+(WMD(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords'])))),3)

df_Jobs_and_Candidates.tail()

## 1.5- Utilizando glove en lugar de word2vec para aplicar WDM.

In [33]:
# Load Word Embedding Model
import gensim
from gensim.models.keyedvectors import KeyedVectors

print('gensim version: %s' % gensim.__version__)
#glove_model = gensim.models.KeyedVectors.load_word2vec_format('../model/text/stanford/glove/glove.6B.50d.vec')
glove_model = KeyedVectors.load_word2vec_format('/home/fedricio/Desktop/Embeddings_Utilizados/Glove/glove.6B.50d.txt', binary=False, no_header=True)

gensim version: 4.0.0


In [36]:
CV_text = df_Jobs_and_Candidates.loc[32,'clean_Content_CV']
CV_text

'calonge federico matias oracle consultant page hcm technical consultant federico ha working oracle tools since october march participated erp cloud projects performing reporting tasks working modules ap ar gl since march participating hcm cloud projects performing reporting extractions integrations tasks working modules core nd recruitment ince october h e member artificial intelligence ai committee oracle strong interest data science machine learning last year computer engineering degree studying th year english personal ac ademic programming projects https github com federicocalonge skills summary oracle hcm cloud core recruitment oracle erp cloud accounts receivable ar accounts payable ap general ledger gl main experience points reporting sql bi publisher intermediate - advanced technical documentation extractions api rest hcm extract web services soap basic - intermediate integrations hdl basic - intermediate extra data analysis implementation machine learning algorithms python in

In [65]:
CV_text = df_Jobs_and_Candidates.loc[0,'clean_Content_CV']
CV_text_2 = df_Jobs_and_Candidates.loc[11,'clean_Content_CV']
CV_text_3 = df_Jobs_and_Candidates.loc[32,'clean_Content_CV']
Job_Keyw_text = "bsc msc phd physics mathematics biomedical software engineering computer science years experience developing image processing machine learning products pipelines advantage familiarity web development e g flask serverless deployment e g docker cloud services e g aws entrepreneurial experience passion working small teams peer-reviewed academic publications"
Job_Keyw_text_2 = "football tennis spanish"
texts_to_compare = [Job_Keyw_text,CV_text,CV_text_2,CV_text_3,Job_Keyw_text_2]
texts_tokens = tokenize(texts_to_compare)   #Función definida en las secciones anteriores. 
print(texts_tokens)

[['bsc', 'msc', 'phd', 'physics', 'mathematics', 'biomedical', 'software', 'engineering', 'computer', 'science', 'years', 'experience', 'developing', 'image', 'processing', 'machine', 'learning', 'products', 'pipelines', 'advantage', 'familiarity', 'web', 'development', 'e', 'g', 'flask', 'serverless', 'deployment', 'e', 'g', 'docker', 'cloud', 'services', 'e', 'g', 'aws', 'entrepreneurial', 'experience', 'passion', 'working', 'small', 'teams', 'peerreviewed', 'academic', 'publications'], ['chandler', 'bing', 'chandler', 'bing', 'vit', 'ac', 'education', 'l', 'date', 'birth', 'may', 'languages', 'known', 'hindi', 'english', 'spanish', 'coursework', 'machine', 'learning', 'data', 'visualization', 'perating', 'systems', 'database', 'management', 'system', 'discrete', 'maths', 'skills', 'c', 'java', 'python', 'arduino', 'android', 'html', 'php', 'javascript', 'mysql', 'r', 'studio', 'keras', 'nltk', 'parser', 'sql', 'experience', 'android', 'club', 'coordinator', 'front', 'end', 'develope

In [63]:
texts_tokens[1]   #CV_text

['chandler',
 'bing',
 'chandler',
 'bing',
 'vit',
 'ac',
 'education',
 'l',
 'date',
 'birth',
 'may',
 'languages',
 'known',
 'hindi',
 'english',
 'spanish',
 'coursework',
 'machine',
 'learning',
 'data',
 'visualization',
 'perating',
 'systems',
 'database',
 'management',
 'system',
 'discrete',
 'maths',
 'skills',
 'c',
 'java',
 'python',
 'arduino',
 'android',
 'html',
 'php',
 'javascript',
 'mysql',
 'r',
 'studio',
 'keras',
 'nltk',
 'parser',
 'sql',
 'experience',
 'android',
 'club',
 'coordinator',
 'front',
 'end',
 'developer',
 'technovit',
 'website',
 'team',
 'key',
 'entertainer',
 'friends',
 'projects',
 'good',
 'females',
 'android',
 'application',
 'w',
 'omen',
 'health',
 'safety',
 'emergency',
 'mode',
 'using',
 'android',
 'modules',
 'safest',
 'route',
 'using',
 'image',
 'processing',
 'disease',
 'prediction',
 'using',
 'svm',
 'linear',
 'regression',
 'face',
 'detection',
 'recognition',
 'using',
 'openmp',
 'opencv',
 'openmp',
 'cu

In [66]:
#Imprimimos las distancias entre Job_Keyw_text y los 3 CVs:

tokens_job_keyw = texts_tokens[0]
tokens_CV_1 = texts_tokens[1]
tokens_CV_2 = texts_tokens[2]
tokens_CV_3 = texts_tokens[3]
tokens_job_keyw_2 = texts_tokens[4]

distance_1 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_1)
print('Distance with CV_1= %.4f' % distance_1)

distance_2 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_2)
print('Distance with CV_2= %.4f' % distance_2)

distance_3 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_3)
print('Distance with CV_3= %.4f' % distance_3)

#Esta última comparación no debe dar buen score, ya que las keywords no tienen nada que ver con el CV:
distance_4 = glove_model.wmdistance(tokens_job_keyw_2, tokens_CV_1)
print('Distance keywords_2 with CV_1= %.4f' % distance_4)

print('#'*40)
############################################################
#Ahora aplicamos lo de --> similarity = 1 / (1 + distance):

sim_1 = round((1 / (1 + distance_1)),3)
print('Similarity with CV_1= %.4f' % sim_1)

sim_2 = round((1 / (1 + distance_2)),3)
print('Similarity with CV_2= %.4f' % sim_2)

sim_3 = round((1 / (1 + distance_3)),3)
print('Similarity with CV_3= %.4f' % sim_3)

#Esta última comparación no debe dar buen score, ya que las keywords no tienen nada que ver con el CV:
sim_4 = round((1 / (1 + distance_4)),3)
print('Similarity keywords_2 with CV_1= %.4f' % sim_4)

Distance with CV_1= 0.8981
Distance with CV_2= 0.8440
Distance with CV_3= 0.8419
Distance keywords_2 with CV_1= 1.2049
########################################
Similarity with CV_1= 0.5270
Similarity with CV_2= 0.5420
Similarity with CV_3= 0.5430
Similarity keywords_2 with CV_1= 0.4540


## 1.6- Utilizando spacy en lugar de word2vec o glove para aplicar WDM.

In [40]:
#Ejemplo de la doc oficial modificado: doc oficial: https://spacy.io/universe/project/wmd-relax
#Para esto descargue spacy: >conda install -c conda-forge spacy
#Y el modelo: python -m spacy download en_core_web_lg
import spacy
import wmd

nlp = spacy.load('en_core_web_lg', create_pipeline=wmd.WMD.create_spacy_pipeline)

In [41]:
#Ejemplo:
doc1 = nlp("Politician speaks to the media in Illinois.")
doc2 = nlp("The president greets the press in Chicago.")
doc3 = nlp("The president greets the press in Chicago.")
print(doc1.similarity(doc2))
print(doc2.similarity(doc3))

0.8705018149808114
1.0


In [44]:
#Nuestro caso:

#Preprocesamos el doc_1 a comparar (el doc_2 no hace falta porque son keywords):
doc_1 = df_Jobs_and_Candidates.loc[19,'clean_Content_CV']
doc_1_tokenize = tokenize([doc_1])              #Usando la funcion definida en las secciones previas.
new_text_doc_1 = ' '.join(doc_1_tokenize[0])    #Pasamos la lista a string.
new_text_doc_1                                  #Nuevo texto sin espacios, stop words y demas (ver todo lo que hace 'tokenize')

doc1 = nlp(new_text_doc_1)
doc2 = nlp('data science, machine learning, pandas, python, sql')
print(doc1.similarity(doc2))

0.7440745766755973


In [45]:
#Hacemos lo mismo que arriba solo que ahora nuestras keywords serán otras que no tendrán que ver con el CV:

doc_1 = df_Jobs_and_Candidates.loc[19,'clean_Content_CV']
doc_1_tokenize = tokenize([doc_1])              
new_text_doc_1 = ' '.join(doc_1_tokenize[0])    
new_text_doc_1                                 

doc1 = nlp(new_text_doc_1)
doc2 = nlp('football tennis spanish')  #CAMBIAMOS ESTO.
print(doc1.similarity(doc2))

0.45084626580079523
